# Quadratic Neural Network

A standard neural network models a linear function

```
y = ax + b
```

at each node in the network. This is a first order polynomial

```
y = a*x^1 + b*x^0
```

I experimented with polynomials of higher order

```
y = (a0)*x^0 + (a1)*x^1 + (a2)*x^2 + ... + (an)*x^n
```

motivated by the knowledge that any function that is complex differentiable on an open set can be represented as a power series, but found that cubic and greater terms resulted in very poor accuracy (at least without modification to the network) so in this notebook I will restrict this notebook to quadratic or second order polynomials

```
y = (a0)*x^0 + (a1)*x^1 + (a2)*x^2
```

It should be noted that the power of neural networks lies in their simplicity at each neuron, which allow them to be fast and makes it possible to find a local minima relatively easily. This modification complicates things, but also allows each neuron to take on the value of a curved function rather than simply a straight line.

It is also interesting to note that if we normalize the input layer x to a standard normal distribution (as is fairly typical in machine learning), then x*x will be distributed with a chi squared distribution

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.nn.parameter import Parameter
import math

import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Get Data

In [2]:
#a better test would use a more difficult dataset
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=True)

## Create Quadratic layer

The below module is copied from PyTorch's *Linear* Layer with quadratic weights added

In [3]:
class Quadratic(nn.Module):
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.linear_weight = Parameter(torch.empty((in_features,out_features), **factory_kwargs))
        self.quadratic_weight = Parameter(torch.empty((in_features,out_features), **factory_kwargs))

        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        nn.init.kaiming_uniform_(self.linear_weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.quadratic_weight, a=math.sqrt(5))

        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.linear_weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        x = input@(self.linear_weight)
        x += (input*input)@(self.quadratic_weight)
        #x += 0.001*(input*input*input)@(self.weight_two)
        return x+self.bias


    def extra_repr(self) -> str:
        return f'in_features={self.in_features}, out_features={self.out_features}, bias={self.bias is not None}'



## Create Network

In [4]:
class QuadNet(nn.Module):
    def __init__(self):
        super(QuadNet, self).__init__()
        self.p1 = Quadratic(784,512,device=device)
        self.p2 = Quadratic(512,256,device=device)
        self.p3 = Quadratic(256,256,device=device)
        self.p4 = Quadratic(256,10,device=device)

    def forward(self, x):
        x = F.relu(self.p1(x))
        x = F.relu(self.p2(x))
        x = F.relu(self.p3(x))
        x = self.p4(x)

        return x

## Train

In [5]:
def train_one_epoch(model, lr=0.01):
    loss_fn = nn.CrossEntropyLoss()
    for param in model.parameters():
      param = param.to(device)
    optimizer = optim.Adam(model.parameters(),lr=lr)

    for inputs, labels in train_dataloader:
      optimizer.zero_grad()
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = model.forward(inputs.reshape(-1,784))
      loss = loss_fn(outputs, labels)
      loss.backward()
      optimizer.step()

## Test

In [6]:
def test(model, dataloader = test_dataloader):

    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels=labels.to(device)
            outputs = model.forward(inputs.reshape(-1,784))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct/total

## It works!

In [7]:
model = QuadNet()
model.to(device)
num_epochs = 5
lr=0.001
#set to false if no gpu
for epoch in range(num_epochs):
  lr*=.5
  train_one_epoch(model,lr = lr)
  test_acc = test(model)
  print(f"epoch : {epoch+1},test acc : {test_acc}")

epoch : 1,test acc : 0.9634
epoch : 2,test acc : 0.97
epoch : 3,test acc : 0.9796
epoch : 4,test acc : 0.9815
epoch : 5,test acc : 0.9826
